In [25]:
!pip install langchain-ollama

import os, getpass

from langchain_core.callbacks import CallbackManagerForLLMRun, AsyncCallbackManagerForLLMRun
from langchain_core.outputs import ChatResult


def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

os.environ['OPENAI_API_KEY'] = os.environ.get('OPEN_AI_KEY')

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_ollama import ChatOllama

gpt4o_chat = ChatOpenAI(model="gpt-4o", temperature=0)
gpt35_chat = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)
ollama_chat = ChatOllama(model="deepseek-r1:14b", temperature=0)

In [26]:
from langchain_core.messages import HumanMessage

# Create a message
msg = HumanMessage(content="Say Hello", name="Adam")

# Create a message list
messages = [msg]

#Invoke a model with a list of messages
gpt4o_chat.invoke(messages)
ollama_chat.invoke(messages)

JSONDecodeError: Extra data: line 2 column 1 (char 115)